# Deep Q-Network (DQN)

**Description:** Implementing DQN algorithm on the Robot Navigation Problem.

## Introduction

**Deep Q-Network (DQN)** is a model-free off-policy algorithm for learning discrete actions.

It uses neural networks to apply function approximation, to estimate the Q-function while avoiding convergence problems in the case of continous actions space, states space or both.

It uses a method called replayed memory, which represents a memory buffer for storing the experiences of the agent.

## Problem

We are trying to solve the **Robot Navigation** problem.
In this setting, we can assume taking three actions: [Forward, LeftTurn, RightTurn].


The implementation of the algorithm is carried out in the rest of the file.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mean_squared_error
from matplotlib import pyplot as plt
import gym
import bot3RLNav


2022-11-03 22:49:56.594808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 22:49:56.758359: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-03 22:49:56.789627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/hu

In [2]:

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.n_actions = action_size
        '''
        # "lr" : learning rate
        # "gamma": discount factor
        # "decay_rate": exponential decay rate of the exploration probability
        # "batch_size": size of the sampled experiences to train the DNN
        # "epsilon:" exploration probability
        '''
        self.lr = 0.001
        self.gamma = 0.99
        self.epsilon = 1.0
        self.decay_rate = 0.005
        self.batch_size = 32
        
        # Define memory buffer for experiences storage
        self.memory_buffer= list()
        # Store the last 2000  time steps
        self.max_memory_buffer = 2000
        
        # Create a model having two hidden layers of 24 units
        # The first layer has the size of "state space"
        # The last layer has the size of "actions space"
        self.model = Sequential([
            Dense(units=24,input_dim=state_size, activation = 'relu'),
            Dense(units=24,activation = 'relu'),
            Dense(units=action_size, activation = 'linear')
        ])
        self.model.compile(loss="mse", optimizer = Adam(lr=self.lr))

    def get_action(self, current_state):
        if np.random.uniform(0,1) < self.epsilon:
            return np.random.choice(range(self.n_actions))
        q_values = self.model.predict(current_state)[0]
        return np.argmax(q_values)

    def update_epsilon(self):
        # Update epsilon at episode termination
        self.epsilon = self.epsilon*np.exp(-self.decay_rate)
        print("epsilon:", self.epsilon)

    def store_episode(self, current_state, action, reward, next_state, done):
        self.memory_buffer.append({
            "current_state":current_state,
            "action":action,
            "reward":reward,
            "next_state":next_state,
            "done":done,
        })
        # @ memory buffer size exceeds the max. size, remove the first element (oldest)
        if len(self.memory_buffer) > self.max_memory_buffer:
            self.memory_buffer.pop(0)

    def train(self, batch_size):
        # Select batch size of experience, after shuffling the memory buffer
        np.random.shuffle(self.memory_buffer)
        batch_sample = self.memory_buffer[0:self.batch_size] 

In [ ]:
env = gym.make('bot3RLNav/DiscreteWorld-v0', map_file="data/gray.jpg")
# prev_state = env.reset()
# x = [x.tolist() for x in prev_state.values()]
# print(x)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
n_episodes = 400
max_iter = 500

agent = DQNAgent(state_size, action_size)
total_steps = 0


In [ ]:
for episode in range(n_episodes):
    current_state = env.reset()
    current_state = np.array([current_state])
    for step in range(max_iter):
        total_steps += 1

        action = agent.get_action(current_state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.array([next_state])
        agent.store_episode(current_state, action, reward, next_state, done)

        if done:
            agent.update_epsilon()
            break
        current_state = next_state
  


    if total_steps >= agent.batch_size:
        agent.train(batch_size=agent.batch_size)

